In [73]:
#Primero importamos las librerías que vamos a usar
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OneHotEncoder,OrdinalEncoder

import folium
import plotly.express as px

In [74]:
#Después importamos los datos del archivo CSV que tenemos
df_bmw = pd.read_csv("/Users/fernandopradagorge/Documents/Universidad/Master Data Science and AI/Nuclio Projects/Entregables/Entregable_1_15ENE24/bmw_pricing_v3.csv")

In [46]:
df_bmw.head()

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta
0,NaN,118,140411.0,100.0,2012-02-01,diesel,black,NaN,True,True,False,NaN,True,NaN,True,NaN,11300.0,2018-01-01
1,BMW,M4,13929.0,317.0,NaN,petrol,grey,convertible,True,True,False,NaN,False,True,True,True,69700.0,2018-02-01
2,BMW,320,183297.0,120.0,2012-04-01,diesel,white,NaN,False,False,False,NaN,True,False,True,False,10200.0,2018-02-01
3,BMW,420,128035.0,135.0,NaN,diesel,red,convertible,True,True,False,NaN,True,True,True,NaN,25100.0,2018-02-01
4,BMW,425,97097.0,160.0,NaN,diesel,silver,NaN,True,True,False,False,False,True,True,True,33400.0,2018-04-01


In [47]:
#Una vez comprobado que el DataFrame carga bien, vamos a ver la estructura que tienen los datos.
df_bmw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   marca                        3873 non-null   object 
 1   modelo                       4840 non-null   object 
 2   km                           4841 non-null   float64
 3   potencia                     4842 non-null   float64
 4   fecha_registro               2420 non-null   object 
 5   tipo_gasolina                4838 non-null   object 
 6   color                        4398 non-null   object 
 7   tipo_coche                   3383 non-null   object 
 8   volante_regulable            4839 non-null   object 
 9   aire_acondicionado           4357 non-null   object 
 10  camara_trasera               4841 non-null   object 
 11  asientos_traseros_plegables  1452 non-null   object 
 12  elevalunas_electrico         4841 non-null   object 
 13  bluetooth         

In [77]:
#Cambiamos las columnas fecha_registro y fecha_venta a formato datetime
for i in ["fecha_registro","fecha_venta"]:
    df_bmw[i]=pd.to_datetime(df_bmw[i])

df_bmw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   marca                        3873 non-null   object        
 1   modelo                       4840 non-null   object        
 2   km                           4841 non-null   float64       
 3   potencia                     4842 non-null   float64       
 4   fecha_registro               2420 non-null   datetime64[ns]
 5   tipo_gasolina                4838 non-null   object        
 6   color                        4398 non-null   object        
 7   tipo_coche                   3383 non-null   object        
 8   volante_regulable            4839 non-null   object        
 9   aire_acondicionado           4357 non-null   object        
 10  camara_trasera               4841 non-null   object        
 11  asientos_traseros_plegables  1452 non-null 

In [78]:
#Para una mejor lectura y comprensión de los datos, dividiremos la columna fecha_venta en 3 nuevas según el día, el mes y  el año de venta.
for col in ["fecha_venta"]:
    df_bmw[col + "_MES"] = df_bmw[col].dt.month
    df_bmw[col + "_AÑO"] = df_bmw[col].dt.year

df_bmw.head()

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta,fecha_venta_MES,fecha_venta_AÑO
0,NaN,118,140411.0,100.0,2012-02-01,diesel,black,NaN,True,True,False,NaN,True,NaN,True,NaN,11300.0,2018-01-01,1.0,2018.0
1,BMW,M4,13929.0,317.0,NaT,petrol,grey,convertible,True,True,False,NaN,False,True,True,True,69700.0,2018-02-01,2.0,2018.0
2,BMW,320,183297.0,120.0,2012-04-01,diesel,white,NaN,False,False,False,NaN,True,False,True,False,10200.0,2018-02-01,2.0,2018.0
3,BMW,420,128035.0,135.0,NaT,diesel,red,convertible,True,True,False,NaN,True,True,True,NaN,25100.0,2018-02-01,2.0,2018.0
4,BMW,425,97097.0,160.0,NaT,diesel,silver,NaN,True,True,False,False,False,True,True,True,33400.0,2018-04-01,4.0,2018.0


In [92]:
#Ahora procederemos a quitar las columnas de marca, modelo (por ser irrelevantes para el estudio) y fecha venta por ser repetida:
df_bmw.drop(["modelo", "marca", "fecha_venta"], axis=1, inplace=True)

,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta_MES,fecha_venta_AÑO
0,140411.0,100.0,2012-02-01,diesel,black,NaN,True,True,False,NaN,True,NaN,True,NaN,11300.0,1.0,2018.0
1,13929.0,317.0,NaT,petrol,grey,convertible,True,True,False,NaN,False,True,True,True,69700.0,2.0,2018.0
2,183297.0,120.0,2012-04-01,diesel,white,NaN,False,False,False,NaN,True,False,True,False,10200.0,2.0,2018.0
3,128035.0,135.0,NaT,diesel,red,convertible,True,True,False,NaN,True,True,True,NaN,25100.0,2.0,2018.0
4,97097.0,160.0,NaT,diesel,silver,NaN,True,True,False,False,False,True,True,True,33400.0,4.0,2018.0


In [49]:
#Vamos a buscar los nulos y a quitarlos de enmedio
df_bmw.isnull().sum()

marca                           970
modelo                            3
km                                2
potencia                          1
fecha_registro                 2423
tipo_gasolina                     5
color                           445
tipo_coche                     1460
volante_regulable                 4
aire_acondicionado              486
camara_trasera                    2
asientos_traseros_plegables    3391
elevalunas_electrico              2
bluetooth                       728
gps                               0
alerta_lim_velocidad            728
precio                            6
fecha_venta                       1
dtype: int64

In [50]:
#Y también eliminamos duplicados
df_bmw[df_bmw.duplicated(keep=False)]

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta


In [51]:
df_bmw.drop_duplicates(inplace=True)

In [52]:
df_bmw[df_bmw["marca"].isnull()]

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta
0,NaN,118,140411.0,100.0,2012-02-01,diesel,black,NaN,True,True,False,NaN,True,NaN,True,NaN,11300.0,2018-01-01
8,NaN,Z4,123886.0,125.0,NaN,petrol,black,convertible,True,False,False,False,False,NaN,True,False,6200.0,2018-03-01
11,NaN,325,228000.0,145.0,NaN,diesel,black,NaN,True,True,False,False,True,False,True,NaN,13300.0,2018-03-01
18,NaN,325,205474.0,145.0,2009-05-01,diesel,NaN,NaN,True,True,False,NaN,True,NaN,True,NaN,11000.0,2018-04-01
21,NaN,218,24868.0,100.0,NaN,petrol,grey,convertible,False,True,False,NaN,False,NaN,True,True,31000.0,2018-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4804,NaN,220 Active Tourer,153358.0,140.0,2015-06-01,diesel,grey,van,False,NaN,False,NaN,False,False,True,True,11700.0,2018-02-01
4807,NaN,218 Active Tourer,197276.0,112.0,NaN,diesel,white,van,True,True,False,NaN,False,NaN,True,True,13100.0,2018-02-01
4825,NaN,225 Active Tourer,33300.0,170.0,NaN,petrol,NaN,van,True,True,False,NaN,False,False,False,True,19200.0,2018-05-01
4834,NaN,218 Gran Tourer,47782.0,110.0,NaN,diesel,blue,NaN,False,NaN,False,NaN,False,False,True,True,14900.0,2018-07-01


In [53]:
for i in df_bmw:
         df_bmw[i].isnull
         df_bmw[i].fillna("Otros", inplace=True)
#Con este bucle for eliminamos los elementos nulos de las columnas que vaya encontrando el programa.

/var/folders/fg/y114z21j2zn039z19n0n8cdh0000gn/T/ipykernel_743/2251588063.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Otros' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_bmw[i].fillna("Otros", inplace=True)


In [54]:
#Ahora tenemos que buscar las variables únicas para poder trabajar con ellas, para ello vamos a hacer un análisis univariable
for i in df_bmw.columns:
    print("\n", df_bmw[i].unique(), "\n")


 ['Otros' 'BMW'] 


 ['118' 'M4' '320' '420' '425' '335' '325' 'Z4' '120' '220' '650' '135'
 '218' '318' '430' 'M3' '330' '225' 'M235' '316' '435' '640'
 '640 Gran Coupé' '435 Gran Coupé' 'i8' '630' '430 Gran Coupé' '635' '520'
 'Otros' '525' '530' '535' '320 Gran Turismo' '518' '116' '328' '523' 'X3'
 'M550' '528' '325 Gran Turismo' '125' '318 Gran Turismo' '420 Gran Coupé'
 'i3' '114' '520 Gran Turismo' '530 Gran Turismo' 'M135' '418 Gran Coupé'
 '330 Gran Turismo' '335 Gran Turismo' '730' '123' '535 Gran Turismo'
 '740' '750' 'ActiveHybrid 5' '735' 'M5' 'X5' 'X1' 'X4' 'X6' 'X6 M' 'X5 M'
 'X5 M50' '218 Active Tourer' '218 Gran Tourer' '216' '220 Active Tourer'
 '216 Gran Tourer' '216 Active Tourer' ' Active Tourer'
 '225 Active Tourer' '214 Gran Tourer'] 


 [140411.0 13929.0 183297.0 ... 19633.0 27920.0 195840.0] 


 [100.0 317.0 120.0 135.0 160.0 225.0 145.0 105.0 125.0 130.0 270.0 110.0
 140.0 180.0 85.0 190.0 236.0 240.0 250.0 230.0 309.0 170.0 142.0 155.0
 210.0 'Otros' 122.0 2

In [55]:
for i in df_bmw.columns:
    print("\n", i, df_bmw[i].value_counts, "\n")


 marca <bound method IndexOpsMixin.value_counts of 0       Otros
1         BMW
2         BMW
3         BMW
4         BMW
        ...  
4838      BMW
4839      BMW
4840      BMW
4841      BMW
4842      BMW
Name: marca, Length: 4843, dtype: object> 


 modelo <bound method IndexOpsMixin.value_counts of 0                     118
1                      M4
2                     320
3                     420
4                     425
              ...        
4838      218 Gran Tourer
4839    218 Active Tourer
4840      218 Gran Tourer
4841    218 Active Tourer
4842                  525
Name: modelo, Length: 4843, dtype: object> 


 km <bound method IndexOpsMixin.value_counts of 0       140411.0
1        13929.0
2       183297.0
3       128035.0
4        97097.0
          ...   
4838     39743.0
4839     49832.0
4840     19633.0
4841     27920.0
4842    195840.0
Name: km, Length: 4843, dtype: object> 


 potencia <bound method IndexOpsMixin.value_counts of 0       100.0
1       317.0
2     

In [56]:
#Ahora separamos las variables en Target, Categórica, Booleanas y Numéricas
df_bmw2 = df_bmw.copy()

In [57]:
df_bmw2["precio"].replace("Otros", np.nan, inplace=True)
df_bmw2["precio"] = df_bmw2["precio"].astype("float64")
df_bmw2["km"].replace("Otros", np.nan, inplace=True)
df_bmw2["km"] = df_bmw2["km"].astype("float64")
df_bmw2["potencia"].replace("Otros", np.nan, inplace=True)
df_bmw2["potencia"] = df_bmw2["potencia"].astype("float64")

In [58]:
df_bmw2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   marca                        4843 non-null   object 
 1   modelo                       4843 non-null   object 
 2   km                           4841 non-null   float64
 3   potencia                     4842 non-null   float64
 4   fecha_registro               4843 non-null   object 
 5   tipo_gasolina                4843 non-null   object 
 6   color                        4843 non-null   object 
 7   tipo_coche                   4843 non-null   object 
 8   volante_regulable            4843 non-null   object 
 9   aire_acondicionado           4843 non-null   object 
 10  camara_trasera               4843 non-null   object 
 11  asientos_traseros_plegables  4843 non-null   object 
 12  elevalunas_electrico         4843 non-null   object 
 13  bluetooth         

In [59]:
df_bmw["marca"].value_counts()

marca
BMW      3873
Otros     970
Name: count, dtype: int64

In [60]:
df_bmw2["bluetooth"] = df_bmw2["bluetooth"].astype("bool")
df_bmw2["volante_regulable"] = df_bmw2["volante_regulable"].astype("bool")
df_bmw2["aire_acondicionado"] = df_bmw2["aire_acondicionado"].astype("bool")
df_bmw2["camara_trasera"] = df_bmw2["camara_trasera"].astype("bool")
df_bmw2["asientos_traseros_plegables"] = df_bmw2["asientos_traseros_plegables"].astype("bool")
df_bmw2["elevalunas_electrico"] = df_bmw2["elevalunas_electrico"].astype("bool")
df_bmw2["alerta_lim_velocidad"] = df_bmw2["alerta_lim_velocidad"].astype("bool")

In [61]:
df_bmw2.head()

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta
0,Otros,118,140411.0,100.0,2012-02-01,diesel,black,Otros,True,True,False,True,True,True,True,True,11300.0,2018-01-01
1,BMW,M4,13929.0,317.0,Otros,petrol,grey,convertible,True,True,False,True,False,True,True,True,69700.0,2018-02-01
2,BMW,320,183297.0,120.0,2012-04-01,diesel,white,Otros,False,False,False,True,True,False,True,False,10200.0,2018-02-01
3,BMW,420,128035.0,135.0,Otros,diesel,red,convertible,True,True,False,True,True,True,True,True,25100.0,2018-02-01
4,BMW,425,97097.0,160.0,Otros,diesel,silver,Otros,True,True,False,False,False,True,True,True,33400.0,2018-04-01


In [68]:

target= ["PRECIO"]
def obtener_lista_variables(dataset):
    lista_numericas=[]
    lista_boolean=[]
    lista_categoricas=[]
    for i in dataset:
        if    (dataset[i].dtype.kind=="f" or dataset[i].dtype.kind=="i") and len(dataset[i].unique())!= 2  and i not in target:
              lista_numericas.append(i)
        elif  (dataset[i].dtype.kind=="f" or dataset[i].dtype.kind=="i")  and len(dataset[i].unique())== 2  and i not in target:
              lista_boolean.append(i)
        elif  (dataset[i].dtype.kind=="O")  and i not in target:
              lista_categoricas.append(i)
                
    return lista_numericas, lista_boolean, lista_categoricas

In [69]:
lista_categoricas, lista_boolean, lista_numericas = obtener_lista_variables(df_bmw2)

In [71]:
df_bmw2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   marca                        4843 non-null   object 
 1   modelo                       4843 non-null   object 
 2   km                           4841 non-null   float64
 3   potencia                     4842 non-null   float64
 4   fecha_registro               4843 non-null   object 
 5   tipo_gasolina                4843 non-null   object 
 6   color                        4843 non-null   object 
 7   tipo_coche                   4843 non-null   object 
 8   volante_regulable            4843 non-null   bool   
 9   aire_acondicionado           4843 non-null   bool   
 10  camara_trasera               4843 non-null   bool   
 11  asientos_traseros_plegables  4843 non-null   bool   
 12  elevalunas_electrico         4843 non-null   bool   
 13  bluetooth         